In [1]:
from pprint import pprint

### 0 - Load text file

In [18]:
paths = ["./TED_Talk_Daily.txt", "./snack_daily.txt", "./Joe_Rogan.txt"]
text_corpus = []
episode = []
for path in paths:
    with open(path) as f:
        line = f.readline()
        episode.append(line)
        while line:
            line = f.readline()
            episode.append(line)
    text_corpus.append(''.join(episode))
    episode = []
# pprint (text_corpus)

In [57]:
def split_and_append(bag_of_words, line):
    word_list = line.split(" ")
    for word in word_list:
        raw_word = '%r'%word
        if "\\" in raw_word:
            idx = raw_word.find("\\")
            word = raw_word[:idx]
        if len(word) == 0:
            continue
        bag_of_words.append(word)
    return bag_of_words

### 1 - Exclude all stopwords from the text file

In [20]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
print(stopwords.words('english'))
stoplist = set(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### 2 - Top Frequent words

In [4]:
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

In [5]:
# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [6]:
# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint(processed_corpus)

[['us',
  'black',
  'people',
  'work',
  'us',
  'future',
  'future',
  'work',
  'shares',
  'us',
  'shares',
  'black'],
 ['big', 'big', '2nd', 'home', '2nd'],
 ['company',
  'one',
  'call',
  'company',
  'workouts',
  'one',
  'home',
  'playstation',
  '4',
  'workouts',
  'playstation',
  '4',
  "i've",
  "i've",
  'work',
  'people',
  'call']]


In [9]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(16 unique tokens: ['black', 'future', 'people', 'shares', 'us']...)


In [15]:
pprint(len(dictionary.token2id))

16


### 3 - TF-IDF modeling

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint(bow_corpus)

[[(0, 2), (1, 2), (2, 1), (3, 2), (4, 3), (5, 2)],
 [(6, 2), (7, 2), (8, 1)],
 [(2, 1),
  (5, 1),
  (8, 1),
  (9, 2),
  (10, 2),
  (11, 2),
  (12, 2),
  (13, 2),
  (14, 2),
  (15, 2)]]


In [13]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "black lives matter" string
words = "black lives matter".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(0, 1.0)]


In [22]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(dictionary.token2id))

### 4 - Load keyword-genre mapping from a JSON

In [30]:
import json

with open('../json/keywords.json') as f:
    data = json.load(f)

In [32]:
data['bucket_to_keyword']['disease']

['corona',
 'coronavirus',
 'covid',
 'sick',
 'flu',
 'headache',
 'sneeze',
 'fever']

### 5 - Vote by 10 top genre & Crop top 3 

In [ ]:
scores = {}
for genre in genres:
    scores[genre] += score
    